# COVID ACT NOW 
- toc:true- branch: master
- badges: true
- comments: true
- author: Richard Castro & Ben Miller
- image: images/us_foods.png

In [1]:
#collapse-hide
#IMPORT LIBRARIES
import pandas as pd
import numpy as py
import datetime 
apiKey = '2edb2afbd2734035a9887a3a3ac1c1a2'
from datetime import timedelta
dtt=datetime.datetime.now().strftime("%Y-%m-%d")

In [2]:
#collapse-hide
#IMPORT COVIDACTNOW AND SET DATE RANGE

d_parser= lambda x: pd.datetime.strptime(x, '%Y-%m-%d')
df=pd.read_csv('https://api.covidactnow.org/v2/counties.timeseries.csv?apiKey='+apiKey+'', 
              parse_dates=['date'], date_parser=d_parser)

<ipython-input-2-12c9487817aa>:3: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  d_parser= lambda x: pd.datetime.strptime(x, '%Y-%m-%d')


In [ ]:
#collapse-hide
dff=df
newestDate= dff['date'].max
firstDate=newestDate()-timedelta(days=1)
endDate=firstDate-timedelta(days=30)

In [4]:
#collapse-hide
#clean data from covidactnow

dff=dff.drop(['metrics.infectionRateCI90','metrics.testPositivityRatioDetails', 'metrics.icuHeadroomRatio', 
          'metrics.icuHeadroomDetails', 'actuals.contactTracers',
          'actuals.hospitalBeds.capacity', 'actuals.hospitalBeds.currentUsageTotal',
          'actuals.hospitalBeds.currentUsageCovid', 'country', 'lat', 'long',
          'actuals.hospitalBeds.typicalUsageRate', 'actuals.icuBeds.capacity', 
          'actuals.icuBeds.currentUsageTotal','actuals.icuBeds.currentUsageCovid', 
          'actuals.icuBeds.typicalUsageRate', 'locationId', 'actuals.cases',
          'actuals.deaths', 'actuals.positiveTests', 'actuals.negativeTests',
          'actuals.newCases','metrics.contactTracerCapacityRatio' ], axis=1)
dff=dff.rename(columns={ 'metrics.testPositivityRatio':'tpr',
       'metrics.caseDensity':'cr', 'metrics.infectionRate':'rt'})

In [ ]:
#collapse-hide
dff.to_csv('Output/'+date+'_CovidActNow.csv')

In [7]:
#collapse-hide
df3=dff

In [5]:
#collapse-hide
mFilter=(dff['date']>=endDate)
df3=dff.loc[mFilter]
df3.columns

NameError: name 'endDate' is not defined

In [8]:
#collapse-hide
#IMPORT CENSUS DATA AND CLEAN THE DATA
census=pd.read_csv('Input/CensusPop.csv')
census=census.drop(['state', 'county', 'density', 'area'], axis=1)

In [9]:
#collapse-hide
#IMPORT GOOGLE DATA AND CLEAN IT UP. 
g=pd.read_csv('Input/Mobility.csv')

g=g.drop(['country_region_code', 'country_region', 'sub_region_1', 'sub_region_2',
       'metro_area', 'iso_3166_2_code', 
       'retail_and_recreation_percent_change_from_baseline',
       'grocery_and_pharmacy_percent_change_from_baseline',
       'parks_percent_change_from_baseline',
       'transit_stations_percent_change_from_baseline',
       'workplaces_percent_change_from_baseline'
       ], axis=1)

g=g.rename(columns={'census_fips_code':'fips', 'residential_percent_change_from_baseline':'mobility'})

In [10]:
#collapse-hide
g.date=pd.to_datetime(g.date)
g.date

0        2020-02-15
1        2020-02-16
2        2020-02-17
3        2020-02-18
4        2020-02-19
            ...    
741625   2020-11-30
741626   2020-12-01
741627   2020-12-02
741628   2020-12-03
741629   2020-12-04
Name: date, Length: 741630, dtype: datetime64[ns]

In [1]:
#collapse-hide
#g=g.set_index('date')
g3=g

NameError: name 'g' is not defined

In [ ]:
#collapse-hide
gFilter=(g3['date']>=endDate)
g3=g3.loc[gFilter]
g3

In [12]:
#collapse-hide
dfm=pd.merge(df3, census, on='fips', how='left')
dfm.head()

,date,state,county,fips,tpr,cr,rt,metrics.icuCapacityRatio,riskLevels.overall,pop
0,2020-01-22,AK,Aleutians East Borough,2013,NaN,0.0,NaN,NaN,0,2938.0
1,2020-01-23,AK,Aleutians East Borough,2013,NaN,0.0,NaN,NaN,0,2938.0
2,2020-01-24,AK,Aleutians East Borough,2013,NaN,0.0,NaN,NaN,0,2938.0
3,2020-01-25,AK,Aleutians East Borough,2013,NaN,0.0,NaN,NaN,0,2938.0
4,2020-01-26,AK,Aleutians East Borough,2013,NaN,0.0,NaN,NaN,0,2938.0


In [13]:
#collapse-hide
dfm2=pd.merge(dfm, g3, how='left', on=['fips', 'date'])
dfm2.head()

,date,state,county,fips,tpr,cr,rt,metrics.icuCapacityRatio,riskLevels.overall,pop,mobility
0,2020-01-22,AK,Aleutians East Borough,2013,NaN,0.0,NaN,NaN,0,2938.0,NaN
1,2020-01-23,AK,Aleutians East Borough,2013,NaN,0.0,NaN,NaN,0,2938.0,NaN
2,2020-01-24,AK,Aleutians East Borough,2013,NaN,0.0,NaN,NaN,0,2938.0,NaN
3,2020-01-25,AK,Aleutians East Borough,2013,NaN,0.0,NaN,NaN,0,2938.0,NaN
4,2020-01-26,AK,Aleutians East Borough,2013,NaN,0.0,NaN,NaN,0,2938.0,NaN


In [14]:
#collapse-hide
dfm2.drop(columns=['metrics.icuCapacityRatio', 'riskLevels.overall'], inplace=True)

In [15]:
#collapse-hide
dfm2.columns

Index(['date', 'state', 'county', 'fips', 'tpr', 'cr', 'rt', 'pop',
       'mobility'],
      dtype='object')

In [16]:
#collapse-hide
dfm2['missing']=0
dfm2['score']=0
dfm2['confidence']=0
dfm2['premath-1']=0
dfm2['premath-2']=0
dfm2['premath-3']=0
dfm2['premath-4']=0
dfm2['postmath-1']=0
dfm2['postmath-2']=0
dfm2['postmath-3']=0
dfm2['postmath-4']=0
dfm2['pop_strata']=0
dfm2['daily']=0
dfm2['weekly']=0
dfm2['change']=0
dfm2.columns

Index(['date', 'state', 'county', 'fips', 'tpr', 'cr', 'rt', 'pop', 'mobility',
       'missing', 'score', 'confidence', 'premath-1', 'premath-2', 'premath-3',
       'premath-4', 'postmath-1', 'postmath-2', 'postmath-3', 'postmath-4',
       'pop_strata', 'daily', 'weekly', 'change'],
      dtype='object')

In [17]:
#collapse-hide
dfm2.to_csv('output/'+dtt+'_CountyOutput(all).csv')